In [ ]:
import os
import shutil
from time import sleep
import pandas as pd
import cv2

In [ ]:
total_seconds = 1757

In [ ]:
def create_or_empty_output_frames(directory):
    # Check if the directory exists
    if os.path.exists(directory):
        # Remove all files in the directory
        shutil.rmtree(directory)
    
    # Create the directory
    os.makedirs(directory)


In [ ]:
df = pd.read_csv('frame_analysis.csv')

# Convert the DataFrame to a dictionary with frame_count as the key
# We'll use 'frame_count' as the key and the rest of the columns as the value
frame_dict = df.set_index('frame_count').T.to_dict('dict')

# Print the resulting dictionary


In [ ]:
def extract_key_frames(ssim_threshold, ssim_threshold_top, ssim_threshold_bottom, histogram_similarity_threshold, time_threshold):
    key_frame_ids = []

    # ssim_overall = 1
    # ssim_top = 1
    # ssim_bottom = 1
    # histogram_similarity = 1
    time = 0


    for frame_id, metrics in frame_dict.items():
        if frame_id < 400: continue

        # ssim_overall = metrics["ssim_overall"]
        # ssim_top = metrics["ssim_top"]
        # ssim_bottom = metrics["ssim_bottom"]
        # histogram_similarity = metrics["histogram_similarity"]
        time += 1/30

        # difference_score, where higher is better
        # and lower is more similar

        # so if ssim_threshold is 0.7, and ssim_overall is 1, you end up with negative
        condition_1 = ssim_threshold - metrics["ssim_overall"]  
        condition_2 = ssim_threshold_top - metrics["ssim_top"]  
        condition_3 = ssim_threshold_bottom - metrics["ssim_bottom"] 
        condition_4 = histogram_similarity_threshold - metrics["histogram_similarity"]
        # so if time_threshold is 1, and time is 0.3
        condition_5 = time - time_threshold 


        if condition_1 + condition_2 + condition_3 + condition_4 + condition_5 > 1:
            key_frame_ids.append(frame_id)
            # ssim_overall, ssim_top, ssim_bottom, histogram_similarity, time = 1,1,1,0
            time = 0

    return key_frame_ids

In [ ]:
def extract_frames(video_path, frame_numbers, output_folder):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    # Check if the video opened successfully
    if not cap.isOpened():
        print("Error: Could not open video file.")
        return

    # Ensure the output directory exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Frame counter
    frame_count = 0

    # Extract specific frames
    extracted_frames = []
    while True:
        # Read the next frame from the video
        ret, frame = cap.read()

        # If the frame could not be read, break the loop
        if not ret:
            break

        # Check if the current frame is in the list of frames to extract
        if frame_count in frame_numbers:
            frame_filename = os.path.join(output_folder, f"frame_{frame_count}.jpg")
            cv2.imwrite(frame_filename, frame)  # Save the frame as a .jpg image
            extracted_frames.append(frame_filename)
            print(f"Extracted frame {frame_count} to {frame_filename}")

        # Increment frame counter
        frame_count += 1

    print(frame_count)

    # Release the video capture object
    cap.release()
    print("Frame extraction completed.")
    return extracted_frames

In [ ]:
# print("ratio\thist_threshold\tssim_threshold\tssim_threshold_caption\tframes")
for i in range(1, 1000):
    # print(i)
    for j in range(1, 1000):
        create_or_empty_output_frames("output_frames")

        # constn = 0.01 * j

        ssim_threshold = 0.30 + 0.76
        ssim_threshold_top = 0.721 + 0.77# - i * 0.001
        ssim_threshold_bottom = 0.739 + 0.77# - j * 0.001
        histogram_similarity_threshold = 0.9999
        time_threshold = 30/60 + 0.76

        if ssim_threshold < 0 or ssim_threshold_top < 0 or ssim_threshold_bottom < 0: break


        frames = extract_key_frames(
            ssim_threshold,
            ssim_threshold_top,
            ssim_threshold_bottom,
            histogram_similarity_threshold,
            time_threshold
        )
        # extract_frames("test.mkv",frames,"output_frames")


        ratio = len(frames) / total_seconds
        print(ratio, j)
        print(len(frames))
        if abs(1-ratio) < 0.03:# or ratio < 0.9 or True:
            print(
                f"{ratio}\t{ssim_threshold}\t{ssim_threshold_top}\t{ssim_threshold_bottom}\t{len(frames)}"
            )
            break
        break
    break
        